# Funções Globais

In [1]:
import numpy as np
import sympy as sp
from tqdm import tqdm
import plotly.graph_objects as go

def gradiente_simbolico(xs_simbolicos, objetivo_simbolico):
    return sp.Array([objetivo_simbolico.diff(x) for x in xs_simbolicos])

def funcao_gradiente(variaveis_simbolicas, gradiente_simbolico):
    return sp.lambdify(variaveis_simbolicas, gradiente_simbolico, 'numpy')

def metodo_gradiente(funcao_gradiente_numerico, x, c, alpha=0.0001, eps=0.001, max_iter=10000):
    x_copia = x.copy()
    print(c)
    for _ in range(max_iter):
        gradiente = funcao_gradiente_numerico(*x_copia, c)
        if np.linalg.norm(gradiente) < eps:
            break
        x_copia -= alpha * gradiente

    return x_copia

def penalidade(tipo, variaveis_simbolicas, xs_simbolicos, ponto_inicial, penalizacao_simbolico, fator, eps=0.01, max_iter=10000):
    x = np.array(ponto_inicial).astype("float64")
    lista_x = [x]
    
    if tipo == "externa":
        c = 0.001
    elif tipo == "interna":
        c = 512
        
    gradiente_sym = gradiente_simbolico(xs_simbolicos, penalizacao_simbolico)
    funcao_grad_numerico = funcao_gradiente(variaveis_simbolicas, gradiente_sym)

    for _ in tqdm(range(max_iter)):
        novo_x = metodo_gradiente(funcao_grad_numerico, x, c)
        lista_x.append(novo_x)

        if np.linalg.norm(x - novo_x) < eps:
            break

        x = novo_x
        
        if tipo == "externa":
            c *= fator
        elif tipo == "interna":
            c /= fator

    return x, np.array(lista_x)

def plotar_funcao_objetivo(f, z_func, xs, nome):
    x_range = (-10, 10)
    y_range = (-10, 10)
    # Define the space for x1 and x2
    
    x1 = np.linspace(*x_range, 100)
    x2 = np.linspace(*y_range, 100)
    x1, x2 = np.meshgrid(x1, x2)
    x3 = z_func(x1, x2)  # apply the constraint

    # Calculate the values of the function for the grid of points
    Z = f(x1, x2, x3)

    # Separate the points
    xs_first = xs[0]
    xs_last = xs[-1]
    xs_middle = xs[1:-1]

    Z_points_first = f(*xs_first)
    Z_points_last = f(*xs_last)
    Z_points_middle = np.array([f(*x) for x in xs_middle])

    # Create the plot
    fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                          go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                       mode='markers', marker=dict(size=6, color='green')),
                          go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                       mode='markers', marker=dict(size=6, color='red')),
                          go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                       mode='markers', marker=dict(size=10, color='blue'))])
    fig.update_layout(scene=dict(
                        xaxis_title='X1',
                        yaxis_title='X2',
                        zaxis_title='Z'),
                      title_text="Função Objetivo 3D Interativa")

    fig.write_html(f"{nome}.html")

    fig.show()


# 1A

In [5]:
# Defina as variáveis simbólicas
x1, x2, x3, c = sp.symbols('x1 x2 x3 c')
variaveis_simbolicas = sp.Array([x1, x2, x3, c])
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-2, 0, 0], [0, np.sqrt(5), 0], [1, 0, 0], [0, 1, 1], [-1, 0, -1]])
b_simbolico = sp.Matrix([7, 2, 7, -2, 5])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrição de igualdade
h = sp.Matrix([x1 + x2 + x3 - 1])

# Psi minusculo
psi = h.dot(h)

# Função de penalização
P = objetivo_simbolico + c * psi

psi_lambdificado = sp.lambdify([x1, x2, x3], psi, "numpy")

x0 = [1, 1, 1]

x_ponto, x_lista = penalidade("externa", variaveis_simbolicas, xs_simbolicos, x0, P, 2)

  0%|          | 2/10000 [00:00<16:19, 10.20it/s]

0.001
0.002


  0%|          | 4/10000 [00:00<18:58,  8.78it/s]

0.004
0.008
0.016


  0%|          | 6/10000 [00:00<18:15,  9.12it/s]

...

8.192
16.384
32.768


  0%|          | 18/10000 [00:02<21:11,  7.85it/s]

65.536
131.072


  0%|          | 19/10000 [00:02<20:45,  8.01it/s]

262.144
524.288


In [6]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2, x3):
    return objetivo(x1, x2, x3)

# Defina o espaço para x1, x2 e x3
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)
x3 = 1 - x1 - x2  # de acordo com a restrição

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2, x3)

xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("1a.html")

fig.show()


# 1B

In [9]:
# Defina as variáveis simbólicas
x1, x2, x3, t = sp.symbols('x1 x2 x3 t')
variaveis_simbolicas = sp.Array([x1, x2, x3, t])
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-2, 0, 0], [0, np.sqrt(5), 0], [1, 0, 0], [0, 1, 1], [-1, 0, -1]])
b_simbolico = sp.Matrix([7, 2, 7, -2, 5])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrição de igualdade
g1x = x1 + x2 + x3 - 1
g2x = -g1x

# g = sp.Matrix([g1x, g2x])

# Psi minusculo, barreira
psi = -(sp.log(-g1x) + sp.log(-g2x))

# Função de penalização
B = objetivo_simbolico + t * psi

x0 = [1, 1, 1]

x_ponto, x_lista = penalidade("interna", variaveis_simbolicas, xs_simbolicos, x0, B, 2)

  0%|          | 1/10000 [00:00<25:19,  6.58it/s]

512
256.0


  0%|          | 3/10000 [00:00<22:19,  7.47it/s]

128.0
64.0


  0%|          | 6/10000 [00:00<21:45,  7.65it/s]

...

0.5
0.25


  0%|          | 13/10000 [00:01<22:12,  7.49it/s]

0.125
0.0625


  0%|          | 14/10000 [00:01<23:36,  7.05it/s]

0.03125


In [10]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2, x3):
    return objetivo(x1, x2, x3)

# Defina o espaço para x1, x2 e x3
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)
x3 = 1 - x1 - x2  # de acordo com a restrição

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2, x3)

xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("1b.html")

fig.show()


# 2B

In [11]:
# Defina as variáveis simbólicas
x1, x2, c = sp.symbols('x1 x2 c')
variaveis_simbolicas = sp.Array([x1, x2, c])
xs_simbolicos = sp.Array([x1, x2])

objetivo_simbolico = 2*x1 + 3*x2 + 4*x1**2 + 2*x1*x2 + x2**2

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrição de igualdade
g1x = x1 - x2
g2x = x1 + x2 - 4
g3x = x1 - 3

# Psi minusculo
psi = sp.Max(0, -g1x)**2 + sp.Max(0, g2x)**2 + sp.Max(0, -g3x)**2

# Função de penalização
P = objetivo_simbolico + c * psi

x0 = [18, -24]

x_ponto, x_lista = penalidade("externa", variaveis_simbolicas, xs_simbolicos, x0, P, 2)

  0%|          | 0/10000 [00:00<?, ?it/s]

0.001


  0%|          | 1/10000 [00:02<5:40:47,  2.04s/it]

0.002


  0%|          | 2/10000 [00:04<5:34:58,  2.01s/it]

...

  0%|          | 20/10000 [00:40<5:39:25,  2.04s/it]

1048.576


  0%|          | 21/10000 [00:42<5:40:27,  2.05s/it]

2097.152


  0%|          | 21/10000 [00:44<5:54:37,  2.13s/it]


In [12]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("2b.html")

fig.show()


# 2C

In [13]:
# Defina as variáveis simbólicas
x1, x2, c = sp.symbols('x1 x2 c')
variaveis_simbolicas = sp.Array([x1, x2, c])
xs_simbolicos = sp.Array([x1, x2])

objetivo_simbolico = 2*x1 + 3*x2 + 4*x1**2 + 2*x1*x2 + x2**2

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrição de igualdade
g1x = x1 - x2
g2x = x1 + x2 - 4
g3x = x1 - 3

# Psi minusculo
psi = psi = -(sp.log(-g1x) + sp.log(-g2x) + sp.log(-g3x))

# Função de penalização
P = objetivo_simbolico + c * psi

x0 = [18, -24]

x_ponto, x_lista = penalidade("interna", variaveis_simbolicas, xs_simbolicos, x0, P, 2)

  0%|          | 2/10000 [00:00<17:02,  9.78it/s]

512
256.0


  0%|          | 4/10000 [00:00<19:34,  8.51it/s]

128.0
64.0


  0%|          | 6/10000 [00:00<19:22,  8.60it/s]

...

2.0
1.0
0.5


  0%|          | 13/10000 [00:01<17:06,  9.73it/s]

0.25
0.125
0.0625


  0%|          | 14/10000 [00:01<19:31,  8.52it/s]

0.03125


In [14]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("2c.html")

fig.show()
